In [1]:
import sys

sys.path.append("../")

In [20]:
import pandas as pd

from classifier.constants import (
    TENNIS_ITEMS,
    FOOTBALL_ITEMS,
    BASKETBALL_ITEMS,
    WINTER_SPORT_ITEMS,
    VOLLEYBALL_ITEMS,
    HOCKEY_ITEMS,
    ATHLETICS_ITEMS,
    ESPORT_ITEMS,
    BOARDGAMES_ITEMS,
    MOTOSPORT_ITEMS,
    AUTOSPORT_ITEMS,
    EXTREME_ITEMS,
    MARTIAL_ARTS_ITEMS,
    CATEGORY_TO_ITEMS
)
from tqdm import tqdm
from collections import Counter
from classifier.voter import Voter
from classifier.factory import build_evaluator
from classifier.preprocessor import Preprocessor

In [3]:
tqdm.pandas()

In [4]:
df = pd.read_csv("../data/train.csv")

In [5]:
preprocessor = Preprocessor(
    do_stem=False,
    remove_stop_words=True, 
    black_list=r"\b.*id.*\b",
    remove_single_letter_token=True,
    max_token_length=25,
)

In [6]:
df["text_preprocessed"] = df["text"].progress_apply(preprocessor._preprocess)

100% 38740/38740 [00:14<00:00, 2657.33it/s]


In [7]:
voter = Voter(category_to_items=CATEGORY_TO_ITEMS, default_category="unknown", default_rank=1)

In [8]:
categories, ranks = voter.vote(df.text_preprocessed)

100% 38740/38740 [00:17<00:00, 2156.49it/s]


In [10]:
df["rank"]     = ranks
df["category"] = categories

In [11]:
df

,oid,category,text,text_preprocessed,rank
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...,волшебные фото виктория поплавская евгениямедв...,2
1,503385563,unknown,Возвращение в подземелье Треша 33 Эйфория тупо...,возвращение подземелье треша эйфория тупости ж...,1
2,146016084,hockey,Лучшие чешские вратари – Доминик Доминатор Гаш...,лучшие чешские вратари доминик доминатор гашек...,2
3,933865449,unknown,Rtokenoid Warhammer40k валрак решил нас подкор...,wаrhаmmеr40к валрак решил подкормить сильно св...,1
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...,шестеркин затаскивает рейнджерс финал восточно...,4
...,...,...,...,...,...
38735,910636962,unknown,8 битная буря снова накрыла пикселями автомоби...,битная буря снова накрыла пикселями автомобиль...,1
38736,669736851,unknown,Ира Сидоркова объясняет как сказалась на ее ма...,ира сидоркова объясняет сказалась машине резул...,1
38737,558919241,unknown,24 я ракетка мира хорват Марин Чилич обыграл и...,ракетка мира хорват марин чилич обыграл испанц...,1
38738,776944963,basketball,Стал известен календарь мужской сборной России...,стал известен календарь мужской сборной россии...,2


## Submit

In [12]:
hidden = pd.read_csv("../data/test.csv")

In [13]:
hidden["text_preprocessed"] = hidden["text"].progress_apply(preprocessor._preprocess)

100% 26260/26260 [00:13<00:00, 1997.92it/s]


In [14]:
categories, ranks = voter.vote(hidden.text_preprocessed)

100% 26260/26260 [00:16<00:00, 1635.64it/s]


In [15]:
hidden["rank"]     = ranks
hidden["category"] = categories

In [17]:
hidden

,oid,text,text_preprocessed,rank,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,спочно сообщество продается 1300р покупкой пиш...,1,unknown
1,452466036,Естественное восстановление после тяжелой трен...,естественное восстановление тяжелой тренировки...,1,unknown
2,161038103,Тема нарядов продолжается Одна из британских ж...,тема нарядов продолжается одна британских журн...,1,unknown
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...,привет избранный спрашиваешь происходит ответ ...,1,unknown
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...,король пятисотников деле андрей рублев успешно...,1,unknown
...,...,...,...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...,выиграй коллекционный пазл оf участия поставь ...,1,unknown
26256,279369911,Волейбол от первого лица Егора Пупынина переко...,волейбол первого лица егора пупынина перекочев...,1,volleyball
26257,600699419,Вы были когда нибудь на свидании где вам задав...,свидании задавали ооочень вопросов знаем насто...,1,unknown
26258,560223506,ТОП 20 самых эффективных общефизических упражн...,топ самых эффективных общефизических упражнени...,1,unknown


In [18]:
hidden.category.value_counts()

unknown         18847
athletics        1195
boardgames        948
autosport         881
basketball        749
hockey            657
volleyball        605
martial_arts      601
tennis            481
football          371
esport            308
motosport         257
winter_sport      191
extreme           169
Name: category, dtype: int64

In [21]:
result = hidden.groupby(["oid"]).apply(lambda df: Counter(df.category).most_common(1)[0][0]).rename("category").reset_index()

In [27]:
result[result.category != "unknown"].to_csv("../data/submission.csv", index=False)

In [16]:
hidden.to_csv("../data/submission_reg_exp.csv", index=False)